In [3]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
url=r'D:\Chatbot\Final_Sentiment analysis.xlsx'
df=pd.read_excel(url)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['Reports_clean'] = df['Reports'].apply(preprocess)
df['Resolution_clean'] = df['Resolution'].apply(preprocess)

In [9]:
df.columns

Index(['Genre', 'Reports', 'Age', 'Year', 'Count', 'Gender', 'Nationality',
       'Resolution', 'Student Name', 'Ticket ID', 'Student_ID', 'GPA',
       'priority', 'status', 'report_time', 'report_hour', 'response_time',
       'response_days', 'Gmail', 'Feedback', 'Resolution_clean',
       'Feedback_clean', 'Resolution_sentiment', 'Feedback_sentiment',
       'response_days_norm', 'status_score', 'satisfaction_score',
       'satisfaction_level', 'response_bins', 'Reports_clean'],
      dtype='object')

In [8]:
mode=df['Gmail'].mode()[0]
df['Gmail'].fillna(mode,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_5404\3794570885.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gmail'].fillna(mode,inplace=True)


In [9]:

df['resolution_time'] = pd.to_datetime(df['resolution_time'])
df['report_time'] = pd.to_datetime(df['report_time'])


df['response_time'] = df['resolution_time'] - df['report_time']

In [10]:

df.to_csv("D:/Chatbot/Final_student_sentiment_cleaned.csv", index=False)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Reports_clean'])

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_response(user_input):
    # Preprocess input
    user_input_clean = preprocess(user_input)
    user_vec = tfidf_vectorizer.transform([user_input_clean])

    # Cosine similarity with all existing complaints
    similarity = cosine_similarity(user_vec, tfidf_matrix)
    idx = np.argmax(similarity)  # index of most similar complaint

    print(f"\n🟢 Matched Complaint:\n{df['Reports'].iloc[idx]}")
    print(f"\n🔵 Suggested Resolution:\n{df['Resolution'].iloc[idx]}")
    print(f"\n📊 Similarity Score: {similarity[0][idx]:.3f}")


In [14]:
get_response("I wish the university offered more support with Arabic language courses.")


🟢 Matched Complaint:
I think it would be helpful if the university offered more language courses for students who want to see a new language. This would not only help international students, but too domestic educatee who are concern in language and culture.

🔵 Suggested Resolution:
Introduce a salmagundi of language courses and cultural workshops to encourage language learning and cultural exchange among all students.

📊 Similarity Score: 0.442


In [22]:
!pip install vaderSentiment 


🟢 Matched Complaint:
I think it would be helpful if the university offered more language courses for students who want to learn a new language. This would not only help international students, but also domestic students who are interested in language and culture.

🔵 Suggested Resolution:
Introduce a variety of language courses and cultural workshops to encourage language learning and cultural exchange among all students.

📊 Similarity Score: 0.353


In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - {'not', 'no', 'nor', 'never'}

stemmer = PorterStemmer()

def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())  # Remove punctuation
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

df['Reports_clean'] = df['Reports_clean'].apply(preprocess)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Reports_clean'])


In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    return analyzer.polarity_scores(text)['compound']

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_response(user_input):
    user_clean = preprocess(user_input)
    user_vec = tfidf_vectorizer.transform([user_clean])
    similarity = cosine_similarity(user_vec, tfidf_matrix)

    idx = np.argmax(similarity)
    best_match_score = similarity[0][idx]

    # Sentiment match
    user_sent = get_sentiment(user_input)
    match_sent = get_sentiment(df['Reports'].iloc[idx])

    print(f"\n📊 Similarity Score: {best_match_score:.3f}")
    print(f"💬 Sentiment Match: {user_sent:.2f} vs. {match_sent:.2f}")

    if abs(user_sent - match_sent) > 0.5:
        print("⚠️ Similar text found, but sentiment mismatch. Escalating to human.")
    else:
        print(f"\n🟢 Matched Complaint:\n{df['Reports'].iloc[idx]}")
        print(f"\n🔵 Suggested Resolution:\n{df['Resolution'].iloc[idx]}")


In [19]:
get_response("I wish the university offered more support with Arabic language courses.")


📊 Similarity Score: 0.323
💬 Sentiment Match: 0.69 vs. 0.46

🟢 Matched Complaint:
The university should offer more support to non-traditional students seeking internships or jobs.

🔵 Suggested Resolution:
Expand corporate mentorship programs that give students exposure to industry-specific career paths.


In [28]:
!pip install sentence-transformers

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/212.5 MB 32.8 MB/s eta 0:00:07
   ---------------------------------------- 1.6/212.5 MB 6.5 MB/s eta 0:00:33
   ---------------------------------------- 2.4/212.5 MB 5.2 MB/s eta 0:00:41
    --------------------------------------- 3.4/212.5 MB 4.7 MB/s eta 0:00:45
    --------------------------------------- 4.2/212.5 MB 4.5 MB/s eta 0:00:47
    --------------------------------------- 5.0/212.5 MB 4.4 MB/s eta 0:00:48
   - -------------------------------------- 5.8/212.5 MB 4.3 MB/s eta 0:00:49
   - -------------------------------------- 6.6/212.5 MB 4.2 MB/s eta 0:00:49
   - -------------------------------------- 7.6/212.5 MB 4.2 MB/s eta 0:00:49
   - -------------------------------------- 8.4/212.5 MB 4.2 MB/s eta 0:00:50
   - -------------------------------------- 9.2/212.5 MB 4.1 MB/s eta 0:00:50
   - -------------------------------------- 10.0/212.5 MB 4.1 MB/s eta

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Create embeddings for all complaints in your dataset
df['complaint_embedding'] = df['Reports_clean'].apply(lambda x: model.encode(x))

In [22]:
def get_response(user_input):
    # Encode user input
    
    user_embedding = model.encode(user_input)

    # Compute cosine similarity
    similarities = df['complaint_embedding'].apply(lambda x: cosine_similarity([user_embedding], [x])[0][0])

    # Get the best match
    best_idx = similarities.idxmax()
    best_score = similarities[best_idx]

    # Threshold to ensure relevance
    if best_score < 0.45:
        return "❌ Sorry, I couldn't find a relevant suggestion. Please rephrase."

    matched_complaint = df.loc[best_idx, 'Reports_clean']
    suggested_resolution = df.loc[best_idx, 'Resolution_clean']

    return f"""
🟢 Matched Complaint:
{matched_complaint}

🔵 Suggested Resolution:
{suggested_resolution}

📊 Similarity Score: {best_score:.3f}
"""

In [23]:
print(get_response("I wish the university offered more support with Arabic language courses."))


🟢 Matched Complaint:
love studi abroad program offer univers theyv return chanc see global experi new cultur earn acknowledg toward degre

🔵 Suggested Resolution:
proceed promote expand study abroad program provide student opportunity explore planetary earn college credit

📊 Similarity Score: 0.542



In [66]:
import pandas as pd



# Save to a new CSV file
df.to_excel("D:/Chatbot/Final_student_tickting_cleaned.xlsx", index=False)


In [60]:
from complaint_matcher import ComplaintMatcher

bot = ComplaintMatcher("Final_student_tickting_cleaned.csv")
print(bot.get_response("I feel there should be better Arabic language courses."))


Encoding all complaints... (one time)
{'matched_complaint': 'think would helpful university offered language course student want see new language would help international student domestic educatee concern language culture', 'suggested_resolution': 'introduce salmagundi language course cultural workshop encourage language learning cultural exchange among student', 'similarity_score': 0.605, 'sentiment_match': {'user_input_score': 0.44, 'matched_complaint_score': 0.7}}


In [61]:
print(bot.get_response("My university account is not working."))

{'matched_complaint': 'ive attempt update personal information university website keep glitching cant fetch customer service frustrating info important communication academic purpose', 'suggested_resolution': 'fix website glitch improve customer serving assist student updating personal information', 'similarity_score': 0.527, 'sentiment_match': {'user_input_score': 0.0, 'matched_complaint_score': -0.27}}


In [24]:

from textblob import TextBlob


In [25]:
def correct_text(text):
    corrected = TextBlob(text).correct()
    return str(corrected)


In [26]:
def get_response(user_input):
    # Encode user input
    corrected_input = correct_text(user_input)
    user_embedding = model.encode(corrected_input)

    # Compute cosine similarity
    similarities = df['complaint_embedding'].apply(lambda x: cosine_similarity([user_embedding], [x])[0][0])

    # Get the best match
    best_idx = similarities.idxmax()
    best_score = similarities[best_idx]

    # Threshold to ensure relevance
    if best_score < 0.45:
        return "❌ Sorry, I couldn't find a relevant suggestion. Please rephrase."

    matched_complaint = df.loc[best_idx, 'Reports_clean']
    suggested_resolution = df.loc[best_idx, 'Resolution_clean']

    return f"""
🟢 Matched Complaint:
{matched_complaint}

🔵 Suggested Resolution:
{suggested_resolution}

📊 Similarity Score: {best_score:.3f}
"""

In [27]:
print(get_response("I neeed halp with admision plz"))


🟢 Matched Complaint:
im worri wont abl afford attend univers next semest tuition rate uphold addit

🔵 Suggested Resolution:
leave tax filing assistance student help maximize potential tax benefit

📊 Similarity Score: 0.550



In [11]:
from full_chat import ComplaintMatcher
import warnings
warnings.filterwarnings("ignore", message="Some weights of BertForSequenceClassification")
warnings.filterwarnings("ignore", category=FutureWarning)
bot = ComplaintMatcher("Final_student_sentiment_cleaned.csv")
print(bot.get_response("I feel there should be better Arabic language courses."))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Encoding all complaints... (one time only)
🤖 Hello! How can I help you today?

📂 Predicted Genre: International student experiences (Confidence: 0.532)

Predicted Genre: International student experiences (Confidence: 0.532)

Matched Complaint:
viii ace wish university offered language class tutoring service international student difficult livelihood coursework youre struggling understand language

Suggested Resolution:
offer workshop public verbalize presentation skill help international student build confidence academic professional setting

Similarity Score: 0.828
Sentiment Match:
  - User Input: 0.44
  - Matched Complaint: -0.2
Corrected Input:
I feel there should be better Arabic language courses.



In [ ]:
print(bot.get_response("I feel there should be better Arabic language courses."))

🤖 Hello! How can I help you today?

Matched Complaint:
viii ace wish university offered language class tutoring service international student difficult livelihood coursework youre struggling understand language

Suggested Resolution:
offer workshop public verbalize presentation skill help international student build confidence academic professional setting

Similarity Score: 0.828
Sentiment Match:
  - User Input: 0.44
  - Matched Complaint: -0.2
Corrected Input:
I feel there should be better Arabic language courses.



In [4]:
print(bot.get_response("أشعر أنه ينبغي أن تكون هناك دورات أفضل للغة العربية"))

🤖 Hello! How can I help you today?

شكوى متطابقة:
5 نقدر الجهد البروفيسور Devecate استيعاب International Educatee Conceive يمكن أن يدعم الموارد المتاحة ربما برنامج تبادل لغة البرنامج التعليمي

القرار المقترح:
توسيع خدمة دعم المصطلحات بما في ذلك ممارسة محادثة التدريس مساعدة الطالب الدولي على تحسين الطلاقة

درجة التشابه: 0.792
مباراة المشاعر:
  - إدخال المستخدم: 0.44
  - شكوى متطابقة: 0.66
المدخلات المصححة:
أشعر أنه يجب أن تكون هناك دورات أفضل للغة العربية


In [30]:
print(matcher.get_response("I can not undrstand the lectre"))

{'matched_complaint': 'textbook course helpful poorly written doesnt explain anything clear', 'suggested_resolution': 'mandate faculty training effective grading transparency provide scholar detailed rubric establish reevaluation request system', 'similarity_score': 0.523, 'sentiment_match': {'user_input_score': 0.0, 'matched_complaint_score': 0.16}, 'corrected_input': 'I can not understand the lecture'}


In [31]:
print(matcher.get_response("i need notes and questations about the lecture"))

{'matched_complaint': 'really enjoy assist lecture talk campus great way ascertain new thing expand knowledge beyond coursework', 'suggested_resolution': 'organize lecture talk variety topic provide student opportunity expand knowledge', 'similarity_score': 0.517, 'sentiment_match': {'user_input_score': 0.0, 'matched_complaint_score': 0.87}, 'corrected_input': 'i need notes and questions about the lecture'}
